In [10]:
from scipy import fft
from scipy.io import wavfile
import scipy.signal as ssig
import sounddevice as sd
import numpy as np
import librosa

In [ ]:
wav = wavfile.read(filename="hw2assets/collectathon.wav")

In [18]:
print(data)

[-11.5 -11.5]


In [2]:
wav, sample_rate = librosa.load(path="hw2assets/collectathon.wav", mono=True, dtype=np.float32)
if wav.ndim == 1:
    print("Mono")
else:
    print("Stereo")

Mono


In [3]:
sd.play(data=wav, samplerate=sample_rate)
sd.sleep(2000) # sleep for 2 seconds to just test play
sd.stop() # stop play once sleep is done.

In [4]:
blockSizes = [64] #, 256, 512, 1024, 4096]

low_bin = None
med_bin = None
high_bin = None

In [5]:
#FFT

In [6]:
#Filter

In [ ]:
for blockSize in blockSizes:
    # Call FFT and Filter, then sum them together (avg? rms? peak?)
    # Combine some how that makes sense from energy perspective
    fft.irf
    
    print(f"Playing audio with block size: {blockSize}")
    sd.play(data=wav_blocks, samplerate=sample_rate)
    wavfile.wr
    # sd.sleep(2000)
    # sd.stop()

Playing audio with block size: 64
